# IPython Async Demo Notebook

This notebook demonstrates the usage of the `ipython-async` package, which allows you to run cells asynchronously across different shells and environments.
This can be usefull, when wanting to run blocking commands, like server and databases, while still wanting to to interact with them in the following cells. 

The package includes four cell magics for jupyter:
- asynccmd (Windows)
- asyncpowershell (Windows)
- asyncbash (Unix)
- asyncpython (any)

This allows you to start any kind of processes from within the notebook, while still interacting with them.

## Simple HTTP Server Example

Let's start a simple server and send it and http request:

In [32]:
%%asynccmd
python -m http.server 8000

Serving HTTP on :: port 8000 (http://[::]:8000/) ...
::1 - - [09/Mar/2025 22:26:17] "GET / HTTP/1.1" 200 -


<Thread(Thread-23 (run_command), started daemon 32896)>

The request can come from a simple cell like:

In [33]:
!curl http://localhost:8000

<!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Directory listing for /</title>
</head>
<body>
<h1>Directory listing for /</h1>
<hr>
<ul>
<li><a href="env.yml">env.yml</a></li>
<li><a href="example.ipynb">example.ipynb</a></li>
</ul>
<hr>
</body>
</html>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   277  100   277    0     0  89210      0 --:--:-- --:--:-- --:--:-- 92333


The remainder of the notebook will introduce a brief introduction of usages

## Installation

Install the package through PyPi. Also make sure you run the notebooks using the conda env supplied in the env.yml located in this directory.

In [ ]:
# Uncomment and run this cell to install ipython-async
# !pip install ipython-async
# conda env create -f env.yml

Now we'll load the extension to make the magic commands available:

In [21]:
%reload_ext ipython_async

## Usage

Let's have a look at how to execute your cells asynchroneously. We can start with Windows command prompt:

### Running Async Command Prompts

The `%%asynccmd` magic by creating background processes in Windows command prompt. To see the effects, let's first run the asynchroneous version, followed by the jupyter notebooks default:

In [ ]:
%%asynccmd
echo Starting loop &
FOR %i IN (1 2 3 4 5) DO (
    echo Async processing step %i &
    ping 127.0.0.1 -n 2 >nul
)

Starting loop 
Async processing step 1 
Async processing step 2 
Async processing step 3 
Async processing step 4 
Async processing step 5 

[Command completed successfully]

<Thread(Thread-28 (run_command), started daemon 28152)>

In [36]:
%%cmd
@echo off >nul & 
echo Starting loop &
FOR %i IN (1 2 3 4 5) DO ( echo Processing step %i & ping 127.0.0.1 -n 2 >nul )

Microsoft Windows [Version 10.0.22631.4974]
(c) Microsoft Corporation. All rights reserved.

(ipython-async-dev) c:\Users\acisse\OneDrive - bridgingIT-Gruppe\003_CodeWorkspace\003_private\ipython-async\usage>@echo off >nul & 
echo Starting loop &
Starting loop 
FOR %i IN (1 2 3 4 5) DO ( echo Processing step %i & ping 127.0.0.1 -n 2 >nul )
Processing step 1 
Processing step 2 
Processing step 3 
Processing step 4 
Processing step 5 


As you should have observed, the asynchroneous version continued executing and printing, while the synchroneous version was already running.

### Running Async Power Shells

Same game for power shell commands:

In [5]:
%%asyncpowershell
# Create and process an array
$numbers = 1..5
foreach ($num in $numbers) {
    Write-Output "Processing number $num"
    Start-Sleep -Seconds 1
}
Write-Output "PowerShell task completed!"

Processing number 1
Processing number 2
Processing number 3
Processing number 4
Processing number 5
PowerShell task completed!

[Command completed successfully]

<Thread(Thread-7 (run_command), started daemon 7184)>

In [6]:
from time import sleep
for i in range(5):
    print(f"iter {i}")
    sleep(0.5)

iter 0
iter 1
iter 2
iter 3
iter 4


Notice that you can continue working in the notebook while the command runs in the background. Let's try another cell:

### Running Async Bash Commands

The `%%asyncbash` magic specifically uses Bash, for execution on Mac and Linux Machines:

In [ ]:
%%asyncbash
# Create a simple array and iterate through it
fruits=("Apple" "Banana" "Cherry" "Date")
for fruit in "${fruits[@]}"; do
    echo "Processing $fruit"
    sleep 0.5
done

<3>WSL (10) ERROR: CreateProcessCommon:559: execvpe(/bin/bash) failed: No such file or directory

[Command failed with exit code: 1]

<Thread(Thread-16 (run_command), started daemon 21160)>

In [ ]:
for i in range(5):
    print(f"iter {i}")
    sleep(0.5)

iter 0
iter 1
iter 2
iter 3
iter 4


### Running Async Python Code

The `%%asyncpython` magic runs simple python code in a separate process asynchronously:

In [1]:
%reload_ext ipython_async
from time import sleep

In [3]:
%%asyncpython
# Simulate some work
print("\nStarting computation...")
for i in range(5):
    print(f"Computing step {i+1}/5 - {'*' * (i+1)}")
    sleep(1)


Starting computation...
Computing step 1/5 - *
Computing step 2/5 - **
Computing step 3/5 - ***
Computing step 4/5 - ****
Computing step 5/5 - *****

[Python code executed successfully]

<Thread(Thread-5 (run_python_code), started daemon 53296)>

Notice, that variables from the context are preserved. In the previous cell, the `sleep` function is impored from the context.

In [4]:
for i in range(5):
    print(f"iter {i}")
    sleep(1)

iter 0
iter 1
iter 2
iter 3
iter 4


## 5. Running Multiple Async Commands Simultaneously

You can run multiple async commands simultaneously, and they'll all execute in parallel:

In [29]:
%%asyncpowershell
echo "Task 1 started"
sleep 5
echo "Task 1 completed"

Task 1 started
Task 1 completed

[Command completed successfully]

<Thread(Thread-19 (run_command), started daemon 28244)>

In [ ]:
%%asynccmd
echo "Task 2 started"
sleep 3
echo "Task 2 completed"

\"Task 2 started\" sleep 3 echo \"Task 2 completed\"

[Command completed successfully]

<Thread(Thread-20 (run_command), started daemon 46152)>

In [ ]:
%%asynccmd
echo "Task 3 started"
sleep 1
echo "Task 3 completed"

\"Task 3 started\" sleep 1 echo \"Task 3 completed\"

[Command completed successfully]

<Thread(Thread-21 (run_command), started daemon 21776)>

## 6. Practical Example: File Processing

Here's a practical example of using `%%asyncpython` to process a large dataset in the background:

In [ ]:
# First, let's create a sample CSV file
import pandas as pd
import numpy as np

# Generate random data
np.random.seed(42)
data = {
    'id': range(1, 10001),
    'value_a': np.random.randn(10000),
    'value_b': np.random.randn(10000),
    'category': np.random.choice(['A', 'B', 'C', 'D'], 10000)
}

df = pd.DataFrame(data)
df.to_csv('sample_data.csv', index=False)
print("Created sample_data.csv with 10,000 rows")

In [ ]:
%%asyncpython
# Process the CSV file in the background
import pandas as pd
import time
import os

print(f"Starting file processing at {time.strftime('%H:%M:%S')}")
print(f"Working directory: {os.getcwd()}")

# Load the data
print("Loading data...")
df = pd.read_csv('sample_data.csv')
print(f"Loaded {len(df)} rows and {len(df.columns)} columns")

# Simulate some heavy processing
print("\nProcessing data...")
for i in range(5):
    print(f"Processing step {i+1}/5")
    time.sleep(1)

# Do some actual calculations
print("\nCalculating statistics...")
stats = df.groupby('category').agg({
    'value_a': ['count', 'mean', 'std', 'min', 'max'],
    'value_b': ['mean', 'std', 'min', 'max']
})

print("\nSummary statistics by category:")
print(stats)

# Calculate correlations
print("\nCalculating correlations...")
corr = df[['value_a', 'value_b']].corr()
print(corr)

# Save results
print("\nSaving results...")
stats.to_csv('stats_results.csv')
print(f"Processing completed at {time.strftime('%H:%M:%S')}")

While the data processing is happening, you can continue working here:

In [ ]:
# Continue with other work while processing happens in the background
print("We can continue working while the file processing runs in the background.")

# Maybe prepare a visualization framework for when results are ready
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.title('Ready for plotting results when processing completes')
plt.text(0.5, 0.5, 'Results will be visualized here', 
         horizontalalignment='center', verticalalignment='center',
         transform=plt.gca().transAxes, fontsize=14)
plt.show()

## 7. Load Results When Ready

Once the background processing is complete, you can load and visualize the results:

In [ ]:
# Check if the results file exists
import os
import time
import pandas as pd

# Wait for file to be available (with timeout)
max_wait = 20  # seconds
start_time = time.time()
file_ready = False

while time.time() - start_time < max_wait:
    if os.path.exists('stats_results.csv'):
        file_ready = True
        break
    print("Waiting for processing to complete...")
    time.sleep(2)

if file_ready:
    # Load and display the results
    stats = pd.read_csv('stats_results.csv')
    print("Results are ready!")
    display(stats.head())
    
    # Visualize some of the results
    category_means = pd.read_csv('stats_results.csv', index_col=0)
    print(category_means.columns)
    
    # Fix the column access for the MultiIndex created by the groupby
    # Depending on how the CSV was saved, you may need to adjust this
    plt.figure(figsize=(10, 6))
    plt.bar(category_means.index, category_means.iloc[:, 1])  # Accessing mean of value_a
    plt.title('Mean of value_a by Category')
    plt.xlabel('Category')
    plt.ylabel('Mean Value')
    plt.show()
else:
    print("Processing is still running or encountered an error. Check the output above.")

## 8. Cleanup

Finally, let's clean up any temporary files we created:

In [ ]:
%%asynccmd
# Clean up temporary files
echo "Cleaning up temporary files..."
if [ -f sample_data.csv ]; then
    rm sample_data.csv
    echo "Removed sample_data.csv"
fi

if [ -f stats_results.csv ]; then
    rm stats_results.csv
    echo "Removed stats_results.csv"
fi

echo "Cleanup completed!"

## Conclusion

The `ipython-async` package allows you to run commands and code asynchronously while continuing to work in your notebook. This is especially useful for:

1. Long-running processes that would otherwise block your notebook
2. Working with multiple shells simultaneously (bash, PowerShell, etc.)
3. Processing data in the background while you prepare visualizations
4. Running system commands without interrupting your workflow

All output is captured and displayed in real-time, making it easy to monitor progress.